<a href="https://colab.research.google.com/github/yahavmh/SVM/blob/master/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import cv2
import numpy as np
import os
import pandas as pd
from PIL import Image
from scipy import misc
import scipy.io as sio
from skimage.color import rgb2gray
from skimage.feature import hog
from sklearn import svm as svm
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from os import path
from xml.etree import ElementTree
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC  
from skimage.io import imread



In [3]:
# function to extract bounding boxes from an annotation file
def extract_boxes(filename):
    # load and parse the file
    tree = ElementTree.parse(filename)
    # get the root of the document
    root = tree.getroot()
    # extract each bounding box
    boxes = []
    labels= []
    
    for box in root.findall('.//bndbox'):
        xmin = int(box.find('xmin').text)
        ymin = int(box.find('ymin').text)
        xmax = int(box.find('xmax').text)
        ymax = int(box.find('ymax').text)
        
        coordinates = [xmin, ymin, xmax, ymax]
        boxes.append(coordinates)
    
    for name in root.iter('object'):
        label = 1
        obj = name.find('name').text
        if obj == 'corn': 
            label = -1
        labels.append(label)
  
    return boxes, labels

## removing background to keep only green objects in image using HSV filter
def background_removal(img):
  hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV) 
  mask = cv2.inRange(hsv, (30, 25, 25), (70, 255,255))

  ## slice the green
  imask = mask>0
  green = np.zeros_like(img, np.uint8)
  green[imask] = img[imask]
  return green 


In [4]:
def read_data(images_path,xml_path):
    ## Reading the images and labels file and distributing them to the train and test set
    data = {'images': [],'class': []}
    os.chdir(images_path)
    num_of_images = len([name for name in os.listdir('.') if os.path.isfile(name)]) ## number of images in dataset
    print( "Number of images in the datase:", num_of_images)

    for i in range(num_of_images):#int(num_of_images/2)): ## iterating on every image and its corresponding xml file
        image_path = images_path + str(i+1) + '.JPG'
        xml_file_path =  xml_path + str(i+1) + '.xml'
       
        if (path.exists(image_path) and path.exists(xml_file_path)): ## make sure we dont upload null
            img = cv2.imread(image_path)  ## Read img
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = background_removal(img)
            boxes, labels = extract_boxes(xml_file_path) ## extract coordinates and labels from xml file
            
            for b in range(len(boxes)):
                box = boxes[b]
                label = labels[b]
                xmin = box[0]
                xmax = box[2]
                ymin = box[1]
                ymax = box[3]
                if((xmax-xmin)>0 and (ymax-ymin)>0): ## for boxes annotated on mistake
                    image = img[ymin:ymax,xmin:xmax]
                    image = cv2.resize(image, (150, 100)) ## VGG16 demands iamge shape of (224,224,3)
                    data['images'].append(image)
                    data['class'].append(label)
    return data

In [5]:
images_path = '/content/drive/My Drive/Colab Notebooks/Images/'
xml_path = '/content/drive/My Drive/פרוייקט גמר/XML/'
print("Reading data...(might take a few minuets)")
data = read_data(images_path,xml_path)
print("Number of annotated object in the dataset:", len(data['class']))

Reading data...(might take a few minuets)
Number of images in the datase: 118
Number of annotated object in the dataset: 33659


In [6]:
def make_hog(data_set): ## HOG function for imgs
    Hog_data_set = {'features': [], 'class': []}
    for i in range(len(data_set['images'])):
        img = data_set['images'][i]
        fd = hog(img, orientations=8, pixels_per_cell=(32, 32),
                    cells_per_block=(1, 1), visualize=False, multichannel=True)
        Hog_data_set['features'].append(fd)
        Hog_data_set['class'].append(data_set['class'][i])
    return Hog_data_set


In [7]:
hog_data = make_hog(data)

In [8]:
## extarcting images and labels from dictionry
featues = np.array(hog_data['features'])
labels = np.array(hog_data['class'])
labels = np.array(labels).reshape(len(labels),1)
data_frame = np.hstack((featues,labels))
np.random.shuffle(data_frame)

In [9]:
percentage = 70
partition = int(len(featues)*percentage/100)
x_train, x_test = data_frame[:partition,:-1],  data_frame[partition:,:-1]
y_train, y_test = data_frame[:partition,-1:].ravel() , data_frame[partition:,-1:].ravel()

In [10]:
clf = svm.SVC(kernel='linear') # Linear Kernel
svm_classifier = clf.fit(x_train, y_train.ravel())

In [11]:
from sklearn.metrics import classification_report,accuracy_score

# Make prediction
y_pred = svm_classifier.predict(x_test)
# Evaluate our model
print("Accuracy: "+str(accuracy_score(y_test, y_pred)))
print('\n')
print(classification_report(y_test, y_pred))

Accuracy: 0.8819568231332937


              precision    recall  f1-score   support

        -1.0       0.91      0.89      0.90      5976
         1.0       0.85      0.87      0.86      4122

    accuracy                           0.88     10098
   macro avg       0.88      0.88      0.88     10098
weighted avg       0.88      0.88      0.88     10098



In [13]:
## preforming gridsearch for optimal hyper-parameters
param_grid = {'C': [0.1,1, 10], 'gamma': [0.1,0.01,0.001],'kernel': ('rbf', 'poly', 'sigmoid','linear')}
grid = GridSearchCV(SVC(),param_grid,refit=True,verbose=1, cv=3)
grid.fit(x_train,y_train)
print(grid.best_estimator_)

Fitting 3 folds for each of 36 candidates, totalling 108 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 108 out of 108 | elapsed: 47.3min finished


SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [14]:
grid_predictions = grid.predict(x_test)
print(confusion_matrix(y_test,grid_predictions))
print(classification_report(y_test,grid_predictions))

[[5551  425]
 [ 323 3799]]
              precision    recall  f1-score   support

        -1.0       0.95      0.93      0.94      5976
         1.0       0.90      0.92      0.91      4122

    accuracy                           0.93     10098
   macro avg       0.92      0.93      0.92     10098
weighted avg       0.93      0.93      0.93     10098

